# Streams and Roofs

In this week's assignment we are going to make some roofline diagrams for some $n$-body problems.

This week's assignment is meant to be run on a node with a Tesla P100 GPU.

A reminder: when you are running a job to complete this week's assignment, you should make sure that you have requested exclusive access to a node, and that you have requested access to all CPU cores of this node.

**Due: Tuesday, September 10, before class**

Let's load in our class module:

In [ ]:
module use $CSE6230_DIR/modulefiles

In [ ]:
module load cse6230

In [ ]:
module list

And verify that we're running where we expect to run:

In [ ]:
nvidia-smi

Great!

Now, about the $n$-body simulations we're going to run: a classical $n$-body simulation has each body, or *particle*, interacting with each other, for $n(n+1)/2$ total interactions.  That hardly matches up to the streaming kernels we've been talking about!  So we're going to simplify a bit.

We are going to simulate $n$ infinitesimal particles circling around an infinitely massive sun at the origin.  In this system, the sun is unmoved, and the particles are not affected by each other.

We're going to normalize our coefficients and say that each particle is an ordinary differential equation with *six* components: three of position $X=(x, y, z)$ and three of velocity $U=(u, v, w)$.  The position, is changed by the velocity, of course, but the velocity changes under acceleration that depends on position:

$$\begin{aligned} \dot{X} &= V \\ \dot{V} &= - \frac{X}{|X|^3}.\end{aligned}$$

To discretize this differential equation, we are going to use a time stepping method called the Verlet leap-frog method, which is good for calculating long simulations of stable orbits.  Given a time step length `dt`, our pseudocode for one time step for one particle looks like the following:

1. `X += 0.5 * dt * V`
2. `R2 = X . X` (dot product)
3. `R = sqrt (R2)`
4. `IR3 = 1. / (R2 * R)`
5. `V -= X * dt * IR3`
6. `X += 0.5 * dt * V`

**Question 1.** Assuming `sqrt` and `div` count for one flop each, and assuming `x, y, z` and `u, v, w` are **double-precision** floating point
numbers, **estimate the arithmetic intensity of a *particle time step***.  You should ignore the time it takes to load `dt`.  Your answer should have units of flops / byte.  Give your answer in a new cell below this one, and show how you arrived at that number.

**Question 2.** Using the peak theoretical **double-precision** flop/s of this node (flop/s on the CPUs and GPU combined), calculated the same way as in the last assignment, and reported peak memory bandwidths from the manufacturers, **estimate the system balance of CPUs and the GPU of this node separately**.  Note that the bandwidth estimate from intel will be for one socket (4 cores) with attached memory, and our node has two such sockets.

Last week, we didn't take the peak flop/s values from the manufacturers at face value, and this week we are not going to take the beak Gbyte/s for granted either.  Last week we used a custom benchmark in our calculations; this week we will use an industry standard: the
[STREAM benchmark](https://www.cs.virginia.edu/stream/ref.html).

We can run the stream benchmark on the CPUs for this assignment with a makefile target:

In [ ]:
make runstream STREAM_N=40000000

The `STREAM_N` argument will control the size of the stream arrays.

**Question 3:** Modify the invocation of `make runstreams` by modifying the values of
`STREAM_N`, `COPTFLAGS` (optimization flags), `OMP_NUM_THREADS` and/or `OMP_PROC_BIND` (the openMP environment variables) to get the largest streaming bandwidth from main memory that you can for this node.

[The OpenMP environment variables were not defined by me in the Makefile: they are environment variables that will be detected by the OpenMP runtime when an OpenMP program begins.  You should put them _before_ the make command, e.g. `OMP_NUM_THREADS=5 make runstream STREAM_N=40000000`]

- Follow the directions in the output of the file and make sure you are testing streaming bandwidth from memory and not from a higher level of cache.
- You should try to get close to the same bandwidth for all tests:

- There are two variables in the openMP environment you should care about, OMP_NUM_THREADS, which is self explanatory, and OMP_PROC_BIND is discussed [here](http://pages.tacc.utexas.edu/~eijkhout/pcse/html/omp-affinity.html).  **You should try to use as few threads as possible** to achieve peak bandwidth.

**Question 4:** What does `OMP_PROC_BIND=close` mean, and why is it a bad choice, not just for this benchmark, but for any streaming kernel?

**Question 5:** I've modified the benchmark, calling it `stream2.c`.  Here's the difference, it's one line of code:

In [ ]:
diff stream.c stream2.c

Copy your options for `runstream` to `runstream2` below.  The reported results should be different: why?

In [ ]:
make runstream2 STREAM_N=1000000

**Question 6:** Now we're going to run stream benchmarks for the GPU.  As above, modify the array size until you believe you are testing streaming bandwidth from memory and not from cache.

In [ ]:
make runstreamcu STREAM_N=1000000

**Question 7 (2 pts):** This is final time we're running a stream benchmark, I promise.  This benchmark is also for the GPU, but instead of the arrays originating in the GPUs memory, they start on the CPUs memory, and must be transfered to the GPU and back.  This mimics a common design pattern when people try to modify their code for GPUs: identify the bottleneck kernel, and try to "offload" it to the GPU, where it will have a higher throughput (once it get's there).  You don't have to modify this run, I just want you to see what bandwidths it reports:

In [ ]:
make runstreamcu2 STREAM_N=1000000

Now, with the three peak bandwidths that we have *computed* (not the reported values from question 2) -- CPU, GPU with arrays on the GPU, and GPU with arrays on the CPU -- and with the theoretical peak flop/s for the CPU and GPU, compute *effective system balances* and create a plot with rooflines for all three balances overlayed.

- The y axis should be absolute Gflop/s, not relative, so we can compare them, and should be labeled "Gflop/s"
- Label with roofline goes with which balance: "CPU", "GPU", "CPU->GPU->CPU"
- The x axis should be in units of "double precision flops / byte"

Save your plot as the jpg `threerooflines.jpg` so that it can embed in the cell below

![Three rooflines](./threerooflines.jpg)

**Question 8 (2 pts):** Remember those particles all the way back in question 1?  Your arithmetic intensity estimate could be placed on the roofline plot for the CPUs, and you could make a judgement about whether the kernel is compute bound or memory bound.

Now let's put it to the test.  The `make runcloud` target simulates `NPOINT` particles orbiting the sun for `NT` time steps.  Because these particles are independent, you can optionally "chunk" multiple time steps for each particle independent of the other particles.  Doing this reduces the number of memory accesses per flop:  each particle stays in register for `NCHUNK` time steps.

Do your best to optimize the throughput of the simulation both in the limit of few particles and many time steps, and in the limit of many particles and few time steps.
Do that by modifying the commands below.

- Make the simulations each run about a second
- Do your best to optimize the compiler flags and the runtime (openMP) environment

Using the outputs of those runs, estimate the floating point efficiency of our particle-time-step kernel: compare the peak flop/s of the CPU, to the product of particle time steps per second and your estimate of the flops per particle time step. and divide by the throughput of particle time steps per second.  Give that effective arithmetic intensity below.

In [ ]:
make runcloud NPOINT=64 NT=1000000 NCHUNK=1

In [ ]:
make runcloud NPOINT=6400000 NT=100 NCHUNK=1